In [ ]:
import numpy as np

import pyfits as pyf

import matplotlib.pyplot as plt

import fitsio

In [ ]:
import sys
sys.path.insert(0, '../')

import sdss_dataset_lib



In [ ]:
sdss_hubble_data = sdss_dataset_lib.SDSSHubbleData()

In [ ]:
portillos_image = np.loadtxt('../../multiband_pcat/Data/idR-002583-2-0136/cts/idR-002583-2-0136-ctsr.txt')

In [ ]:
my_image = sdss_hubble_data.sdss_image.squeeze()

In [ ]:
plt.matshow((my_image[0:100, 0:100].numpy() - (portillos_image - 1044.)) / (portillos_image - 1044.))
plt.colorbar()

In [ ]:
import simulated_datasets_lib

In [ ]:
simulator = simulated_datasets_lib.StarSimulator(psf_fit_file=str(sdss_hubble_data.psf_file), 
                                    slen = sdss_hubble_data.slen, 
                                    sky_intensity = 179.)

In [ ]:
sdss_hubble_data.psf_file

In [ ]:
import torch

In [ ]:
recon_mean = simulator.draw_image_from_params(locs = sdss_hubble_data.locs.unsqueeze(0), 
                                fluxes = sdss_hubble_data.fluxes.unsqueeze(0), 
                                n_stars = torch.Tensor([len(sdss_hubble_data.locs)]).type(torch.LongTensor), 
                                add_noise = False) 

In [ ]:
plt.matshow(sdss_hubble_data.sdss_image.squeeze())
plt.colorbar()

In [ ]:
plt.matshow(recon_mean.squeeze())
plt.colorbar()

In [ ]:
import plotting_utils

In [ ]:
which_bright = sdss_hubble_data.fluxes > 1000.

In [ ]:
### plot some subimages 
f, axarr = plt.subplots(1, 3, figsize=(16, 4))

x0_vec = np.arange(0, 100, 10)
x1_vec = x0_vec

x0 = int(np.random.choice(x0_vec, 1))
x1 = int(np.random.choice(x1_vec, 1))

print([x0, x1])

plotting_utils.plot_subimage(axarr[0], 
                            my_image, 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f)


plotting_utils.plot_subimage(axarr[1], 
                            recon_mean.squeeze(), 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f)

residual = my_image - recon_mean.squeeze()
plotting_utils.plot_subimage(axarr[2], 
                            residual / my_image, 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f, 
                            diverging_cmap = True)

In [ ]:
plt.hist((residual / my_image).flatten(), bins = 100);

# Check out portillos flux units

In [ ]:
results_dir = '../../multiband_pcat/pcat-lion-results/20191017-113443/'

In [ ]:
pcat_catalog = np.loadtxt(results_dir + 'classical_catalog.txt')
    
x1_loc = pcat_catalog[:, 0]
x0_loc = pcat_catalog[:, 2]

fluxes = pcat_catalog[:, 4]

# remove na
is_na = np.isnan(x1_loc) | np.isnan(x1_loc) | np.isnan(fluxes)

x1_loc = x1_loc[~is_na]
x0_loc = x0_loc[~is_na]
fluxes = fluxes[~is_na]

portillos_est_locs = torch.Tensor([x0_loc, x1_loc]).transpose(0,1) / (sdss_hubble_data.slen - 1)
portillos_est_fluxes = torch.Tensor(fluxes)

In [ ]:
# only works if we have the classical catalogue
_locs = portillos_est_locs.unsqueeze(0) 
_fluxes = torch.Tensor(portillos_est_fluxes).unsqueeze(0)
_n_stars = torch.Tensor([len(x0_loc)]).type(torch.LongTensor)
    
portillos_recon_mean = simulator.draw_image_from_params(locs = _locs, 
                                            fluxes = _fluxes,
                                             n_stars = _n_stars,  
                                             add_noise = False) + sdss_hubble_data.sdss_background

plt.matshow(portillos_recon_mean.squeeze()); 
plt.colorbar()

In [ ]:
foo = (portillos_image - 1044.) / 

In [ ]:
### plot some subimages 
f, axarr = plt.subplots(1, 3, figsize=(16, 4))

x0_vec = np.arange(0, 100, 10)
x1_vec = x0_vec

x0 = int(np.random.choice(x0_vec, 1))
x1 = int(np.random.choice(x1_vec, 1))

print([x0, x1])

plotting_utils.plot_subimage(axarr[0], 
                            observed, 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f)


plotting_utils.plot_subimage(axarr[1], 
                            portillos_recon_mean.squeeze(), 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f)

residual = observed - portillos_recon_mean.squeeze()
plotting_utils.plot_subimage(axarr[2], 
                            residual / observed, 
                            None, 
                            sdss_hubble_data.locs[which_bright], 
                            x0, x1, 
                            subimage_slen = 10, 
                            add_colorbar = True, 
                            global_fig = f, 
                            diverging_cmap = True)

In [ ]:
residual / observed

In [ ]:
plt.hist((residual / observed).flatten(), bins = 100);

In [ ]:
plt.hist(torch.log10(sdss_hubble_data.sdss_image.flatten()), bins = 100);

In [ ]:
simulator.psf.max()

In [ ]:
simulator.psf.mean()

In [ ]:
portillos_psf = np.loadtxt('../../multiband_pcat/Data/idR-002583-2-0136/psfs/idR-002583-2-0136-psfr.txt', skiprows=1)

In [ ]:
portillos_psf.max()

In [ ]:
portillos_psf.mean()

In [ ]:
plt.matshow(simulator.psf_og)

In [ ]:
plt.matshow(portillos_psf)